In [12]:
import os

In [13]:
%pwd

'c:\\Users\\deept\\ShopTalk'

In [14]:
os.chdir("../")

In [15]:
%pwd

'c:\\Users\\deept'

In [16]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/forghmc/ShopTalk.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="forghmc"
os.environ["MLFLOW_TRACKING_PASSWORD"]="0749a14e339ee9e48d4d3beeb5d2247a8daeb17b"


In [17]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
   

In [18]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [19]:
class ConfigurationManager:
    
    CONFIG_FILE_PATH = r'C:\Users\deept\ShopTalk\config\config.yaml'
    PARAMS_FILE_PATH = r"C:\Users\deept\ShopTalk\params.yaml"
    SCHEMA_FILE_PATH = r"C:\Users\deept\ShopTalk\schema.yaml"

    def __init__(
        self,
        config_filepath = Path(CONFIG_FILE_PATH),
        params_filepath = Path(PARAMS_FILE_PATH),
        schema_filepath = Path(SCHEMA_FILE_PATH)):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/forghmc/ShopTalk.mlflow",
           
        )

        return model_evaluation_config


In [20]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [21]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_json(self.config.test_data_path, lines=True)
        model = joblib.load(self.config.model_path)

        test_data = pd.read_json(self.config.test_data_path, lines=True)
        print("Columns in test_data:", test_data.columns)  # This will list all column names in test_data

        # Check if the target column exists before attempting to drop it
        if self.config.target_column in test_data.columns:
            test_x = test_data.drop([self.config.target_column], axis=1)
            test_y = test_data[[self.config.target_column]]
        else:
            raise KeyError(f"Column '{self.config.target_column}' not found in test_data")

        

    


In [22]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-04-23 22:57:17,420: INFO: common: yaml file: C:\Users\deept\ShopTalk\config\config.yaml loaded successfully]
[2024-04-23 22:57:17,425: INFO: common: yaml file: C:\Users\deept\ShopTalk\params.yaml loaded successfully]
[2024-04-23 22:57:17,455: INFO: common: yaml file: C:\Users\deept\ShopTalk\schema.yaml loaded successfully]
[2024-04-23 22:57:17,460: INFO: common: created directory at: artifacts]
[2024-04-23 22:57:17,463: INFO: common: created directory at: artifacts/model_evaluation]
Columns in test_data: Index(['brand', 'bullet_point', 'color', 'item_id', 'item_name', 'model_name',
       'model_number', 'model_year', 'product_type', 'style', 'main_image_id',
       'other_image_id', 'item_keywords', 'country', 'marketplace',
       'domain_name', 'node', 'item_dimensions', 'item_weight', 'material',
       'fabric_type', 'color_code', 'product_description', 'spin_id',
       '3dmodel_id', 'pattern', 'finish_type', 'item_shape'],
      dtype='object')
